# A backward pass of Multi-layer Perceptrons

Suppose that we want to build a neural network architecture that has 3 input units, 2 hidden layers of 4 units each and 2 output units.

In [9]:
import numpy as np
np.random.seed(12345)

In [10]:
def initialize(input_dim, hidden1_dim, hidden2_dim, output_dim, batch_size):
    W1 = np.random.randn(hidden1_dim, input_dim) * 0.01
    b1 = np.zeros((hidden1_dim,))
    W2 = np.random.randn(hidden2_dim, hidden1_dim) * 0.01
    b2 = np.zeros((hidden2_dim,))
    W3 = np.random.randn(output_dim, hidden2_dim) * 0.01
    b3 = np.zeros((output_dim,))

    parameters = [W1, b1, W2, b2, W3, b3]
    x = np.random.rand(input_dim, batch_size)
    y = np.random.randn(output_dim, batch_size)

    return parameters, x, y

In [11]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [12]:
def deriv_sigmoid(x):
    #####################################
    #   Computing derivative of sigmoid
    sig_d = sigmoid(x) * (1 - sigmoid(x))
    #####################################
    return sig_d

In [21]:
def forward(parameters, X):
    W1, b1, W2, b2, W3, b3 = parameters

    batch_size = X.shape[1]
    hidden1_dim = W1.shape[0]
    hidden2_dim = W2.shape[0]
    output_dim = W3.shape[0]

    hid_1 = np.zeros((hidden1_dim, batch_size))
    hid_2 = np.zeros((hidden2_dim, batch_size))
    outputs = np.zeros((output_dim, batch_size))

    #####################################
    #   Computing forward pass
    hid_1 = sigmoid(W1.dot(X) + np.tile(np.reshape(b1,(hidden1_dim, 1)), batch_size))
    hid_2 = sigmoid(W2.dot(hid_1) + np.tile(np.reshape(b2,(hidden2_dim, 1)), batch_size))
    outputs = W3.dot(hid_2) + np.tile(np.reshape(b3,(output_dim, 1)), batch_size)
    #####################################

    activations = [X, hid_1, hid_2, outputs]

    return activations

In [22]:
def squared_loss(predictions, targets):
    """ Computes mean squared error

    predictions: (output_dim, batch_size)
    targets: (output_dim, batch_size)

    """

    loss = np.zeros(targets.shape[1])

    #####################
    loss = 0.5 * np.sum((predictions - targets) ** 2, axis = 0)
    #####################

    return np.mean(loss)

In [37]:
def deriv_squared_loss(predictions, targets):
    batch_size = targets.shape[1]

    dloss = np.zeros(targets.shape)
    
    #####################
    dloss = (predictions - targets) / batch_size
    #####################
    
    return dloss

In [38]:
def backward(activations, targets, parameters):

    X, hid_1, hid_2, predictions = activations

    input_dim = X.shape[0]
    hidden1_dim = hid_1.shape[0]
    hidden2_dim = hid_2.shape[0]
    output_dim = predictions.shape[0]

    W1, b1, W2, b2, W3, b3 = parameters

    dW1 = np.zeros((hidden1_dim, input_dim))
    db1 = np.zeros((hidden1_dim,))
    dW2 = np.zeros((hidden2_dim, hidden1_dim))
    db2 = np.zeros((hidden2_dim,))
    dW3 = np.zeros((output_dim, hidden2_dim))
    db3 = np.zeros((output_dim,))

    ##############################
    #   Computing the gradients
    #
    dO = deriv_squared_loss(predictions, targets)
    dW3 = dO.dot(hid_2.T)
    db3 = np.sum(dO, axis = 1)
    dhid_2 = W3.T.dot(dO)
    dy_2 =  deriv_sigmoid(W2.dot(hid_1) + np.tile(np.reshape(b2,(hidden2_dim, 1)), X.shape[1])) * dhid_2
    dW2 = dy_2.dot(hid_1.T)
    db2 = np.sum(dy_2, axis = 1)
    dhid_1 = W2.T.dot(dy_2)
    dy_1 = deriv_sigmoid(W1.dot(X) + np.tile(np.reshape(b1,(hidden1_dim, 1)), X.shape[1])) * dhid_1
    dW1 = dy_1.dot(X.T)
    db1 = np.sum(dy_1, axis = 1)
    ##############################

    grads = [dW1, db1, dW2, db2, dW3, db3]

    return grads

In [39]:
def convert_to_1d_vector(parameters):
    W1, b1, W2, b2, W3, b3 = parameters
    params = np.concatenate([W1.ravel(), b1.ravel(),
                             W2.ravel(), b2.ravel(),
                             W3.ravel(), b3.ravel()], axis=0)

    return params

In [40]:
def convert_to_list(params, input_dim, hidden1_dim, hidden2_dim, output_dim):
    base_idx = 0

    W1 = np.reshape(params[base_idx: base_idx + input_dim * hidden1_dim],
                    (hidden1_dim, input_dim))
    base_idx += input_dim * hidden1_dim

    b1 = params[base_idx: base_idx + hidden1_dim]
    base_idx += hidden1_dim

    W2 = np.reshape(params[base_idx: base_idx + hidden1_dim * hidden2_dim],
                    (hidden2_dim, hidden1_dim))
    base_idx += hidden1_dim * hidden2_dim

    b2 = params[base_idx: base_idx + hidden2_dim]
    base_idx += hidden2_dim

    W3 = np.reshape(params[base_idx: base_idx + hidden2_dim * output_dim],
                    (output_dim, hidden2_dim))
    base_idx += hidden2_dim * output_dim

    b3 = params[base_idx: base_idx + output_dim]

    parameters = [W1, b1, W2, b2, W3, b3]

    return parameters

In [64]:
def eval_numerical_gradient(parameters, gradients, X, Y, loss, eps=1e-7):
    W1, b1, W2, b2, W3, b3 = parameters
    network_structure = [X.shape[0], W1.shape[0], W2.shape[0], W3.shape[0]]

    # convert a list of parameters to a single vector
    params = convert_to_1d_vector(parameters)
    grads = convert_to_1d_vector(gradients)

    n_params = len(params)
    num_grads = np.zeros((n_params,))
    diff = 0.
    
    ##############################
    #   Computing the numerical gradients
    #
    for i in range(n_params):
        temp = params.copy()
        temp[i] += eps
        num_grads[i] = (loss(forward(convert_to_list(temp, X.shape[0], W1.shape[0], W2.shape[0], W3.shape[0]), X)[3], Y) - loss(forward(parameters, X)[3], Y)) / eps
        diff += abs(num_grads[i] - grads[i]) / n_params
    ##############################

    

    return diff, num_grads


In [65]:
input_dim = 3
hidden_dim = 4
output_dim = 2
batch_size = 5


#for _ in range(10):
parameters, X, Y = initialize(input_dim, hidden_dim, hidden_dim, output_dim, batch_size)

activations = forward(parameters, X)

P = activations[-1]

loss = squared_loss(P, Y)
print('Loss: {}'.format(loss))

grads = backward(activations, Y, parameters)

print("Analytic Gradients:")
for i in range(3):
    print("delta W_{}:\n {}".format(i + 1, grads[2 * i]))
    print("delta b_{}:\n {}".format(i + 1, grads[2 * i + 1]))
print("\n")
eps = 1e-5
diff, num_grads = eval_numerical_gradient(parameters, grads, X, Y, squared_loss, eps=eps)
n_grads = convert_to_list(num_grads, input_dim, hidden_dim, hidden_dim, output_dim)
print("Numerical Gradients:")
for i in range(3):
    print("delta W_{}:\n {}".format(i + 1, n_grads[2 * i]))
    print("delta b_{}:\n {}".format(i + 1, n_grads[2 * i + 1]))
print('\nGradient checking: ')
if diff < eps:
    print('\tPassed:', diff)
else:
    print('\tFailed:', diff)    

Loss: 0.4421052327576752
Analytic Gradients:
delta W_1:
 [[-9.23772285e-07  3.52533232e-06  2.24524093e-06]
 [ 3.90191387e-07 -6.98072440e-06 -5.11339794e-06]
 [ 8.92492322e-07 -4.18690300e-06 -2.76151387e-06]
 [-4.80861568e-07 -3.31217684e-06 -2.73519496e-06]]
delta b_1:
 [ 9.97614590e-07 -5.08599597e-06 -1.62716272e-06 -3.85286549e-06]
delta W_2:
 [[ 7.94000557e-05  8.00206970e-05  7.94216697e-05  7.92077596e-05]
 [-4.36906294e-04 -4.39414328e-04 -4.36904288e-04 -4.36625619e-04]
 [-8.94206921e-05 -8.88636841e-05 -8.92775812e-05 -9.02805895e-05]
 [-2.28644338e-04 -2.32267214e-04 -2.28951316e-04 -2.26517312e-04]]
delta b_2:
 [ 0.0001591  -0.00087504 -0.0001786  -0.00045898]
delta W_3:
 [[ 0.04068472  0.04101979  0.04062781  0.04104827]
 [-0.14487681 -0.14607365 -0.14467347 -0.14618082]]
delta b_3:
 [ 0.08145744 -0.2900783 ]


Numerical Gradients:
delta W_1:
 [[-9.23777721e-07  3.52533003e-06  2.24523178e-06]
 [ 3.90193433e-07 -6.98072711e-06 -5.11340414e-06]
 [ 8.92486085e-07 -4.186911

We can see that the difference between the analytic and numerical gradient is almost 0. So we have a correct backpropagation model.